In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_csv("smartphoneinfo.csv", sep = ";")
data = data.drop('idinfo', axis = 1)
data = data.drop('ref', axis = 1) 

for index, row in data.iterrows():
    # smartphone name
    strname = 'Apple' if str(row['smartphonename']).split(' ', 1)[0] == 'iPhone' else str(row['smartphonename']).split(' ', 1)[0]
    namenum = -1
    name = strname.lower()
    if name == 'apple':
        namenum = 0
    elif name == 'asus':
        namenum = 1
    elif name == 'bphone':
        namenum = 2
    elif name == 'honor':
        namenum = 3
    elif name == 'htc':
        namenum = 4
    elif name == 'huawei':
        namenum = 5
    elif name == 'itel':
        namenum = 6
    elif name == 'mobell':
        namenum = 7
    elif name == 'mobiistar':
        namenum = 8
    elif name == 'motorola':
        namenum = 9
    elif name == 'nokia':
        namenum = 10
    elif name == 'oppo':
        namenum = 11
    elif name == 'philips':
        namenum = 12
    elif name == 'samsung':
        namenum = 13
    elif name == 'sony':
        namenum = 14
    elif name == 'vivo':
        namenum = 15
    elif name == 'xiaomi':
        namenum = 16
    data.loc[index, 'smartphonename'] = namenum
    # ram
    data.loc[index, 'ram'] = str(row['ram'])[0:-3]
    # rom
    data.loc[index, 'rom'] = str(row['rom'])[0:-3]
    # cpu brand
    cpu = str(row['cpu']).split(' ')
    cpu[0] = cpu[0].lower()
    cpunum = -1
    
    if cpu[0] == 'exynos' or cpu[0] == 'samsung':
        cpunum = 0
    elif cpu[0] == 'apple':
        cpunum = 1
    elif cpu[0] == 'hisilicon':
        cpunum = 2
    elif cpu[0] == 'snapdragon' or cpu[0] == 'qualcomm':
        cpunum = 3
    elif cpu[0] == 'spreadtrum':
        cpunum = 4
    elif cpu[0] == 'mediatek' or cpu[0][0:2] == 'mt':
        cpunum = 5
    
    data.loc[index, 'cpubrand'] = cpunum
    #camera - front & back
    camerarearpixel = re.findall(r"\d+ MP",str(row['camera'])) 
    
    if camerarearpixel != []:
        data.loc[index, 'camrearpixel'] = str(camerarearpixel[0])[0:-3]
    else:
        data.loc[index, 'camrearpixel'] = '0'
    data.loc[index, 'camrearcount'] = len(camerarearpixel)
    
    camerafrontpixel = re.findall(r"\d+ MP",str(row['selfie']))
    
    if camerafrontpixel != []:
        data.loc[index, 'camfrontpixel'] = str(camerafrontpixel[0])[0:-3]
    else: 
        data.loc[index, 'camfrontpixel'] = '0'
    data.loc[index, 'camfrontcount'] = len(camerafrontpixel)
    
    
    # battery
    data.loc[index, 'battery'] = str(row['battery'])[0:-4]
    # os
    os = str(row['os']).split(' ')
    
    osnum = -1
    for o in os:
        if o.lower() == 'android':
            osnum = 0
            break
        elif o.lower() == 'ios':
            osnum = 1
            break

    data.loc[index, 'os'] = osnum
    #cpuclock
    xn = re.findall('[1-9]x', data.loc[index, 'cpuclock'])
    yn = re.findall('[1-9] nhân', data.loc[index, 'cpuclock'])
    gn = re.findall('\d+\.\d+ GHz', data.loc[index, 'cpuclock'])
    gm = re.findall('\d+\.\d+GHz', data.loc[index, 'cpuclock'])
    if xn:
        data.loc[index,'numcpu1'] = int(xn[0][0:-1])
        if  len(xn) == 2:
            data.loc[index,'numcpu2'] = int(xn[1][0:-1])
        else:
            data.loc[index,'numcpu2'] = 0
    elif yn:
        data.loc[index,'numcpu1'] = int(yn[0][0:-5])
        if  len(yn) == 2:
            data.loc[index,'numcpu2'] = int(yn[1][0:-5])
        else:
            data.loc[index,'numcpu2'] = 0
    else:
        gc = re.findall('\d nhân', data.loc[index, 'cpu'])
        if gc:
            data.loc[index,'numcpu1'] = int(gc[0][0:-5])
            data.loc[index,'numcpu2'] = 0
        else:
            data.loc[index,'numcpu1'] = 1
            data.loc[index,'numcpu2'] = 0
        
    if gn:
        data.loc[index,'clockcpu1'] = float(gn[0][0:-4])
        if len(gn) == 2:
            data.loc[index,'clockcpu2'] = float(gn[1][0:-4])
        else:
            data.loc[index,'clockcpu2'] = 0
    if gm:
        data.loc[index,'clockcpu1'] = float(gm[0][0:-3])
        if len(gn) == 2:
            data.loc[index,'clockcpu2'] = float(gm[1][0:-3])
        else:
            data.loc[index,'clockcpu2'] = 0
    #displayres
    disres = re.search('(\d)+ x (\d)+', str(row['displayres']))
    if disres != None:
        res = disres.group(0).split(' x ')
        if res != []:
            data.loc[index,'displayWidth'] =  min(int(res[0]),int(res[1]))
            data.loc[index,'displayHeight'] =  max(int(res[0]),int(res[1]))
        else: 
            data.loc[index,'displayWidth'] =  -1
            data.loc[index,'displayHeight'] =  -1
    else:
        data.loc[index,'displayWidth'] =  -1
        data.loc[index,'displayHeight'] =  -1
    #brandnew    
    brandnew = re.search('(\d)+\%', str(row['smartphonename']))
    if brandnew is None:
        data.loc[index,'brandnew'] =  1
    else : 
        data.loc[index,'brandnew'] =  float(brandnew.group(0)[0:-1]) / 100
    
    if str(row['provider']) == 'TGDD':
        data.loc[index,'isPortable'] =  1
    else:
        checkPortable = re.search('hãng', str(row['smartphonename']))
        if checkPortable is None:
            data.loc[index,'isPortable'] =  0
        else:
            data.loc[index,'isPortable'] =  1
            
    #provider
    prov = row['provider'].lower()
    if prov == 'tgdd':
        data.loc[index, 'provider'] = 0
    else:
        data.loc[index, 'provider'] = 1
    #price
    data.loc[index, 'price'] = row['price'] / 1000000
    
# delete part
data = data[data.smartphonename != -1]
data = data.drop('cpu', axis = 1)
data = data.drop('cameranumber', axis = 1)
data = data.drop('camera', axis = 1)
data = data.drop('selfie', axis = 1)
data = data.drop('cpuclock', axis = 1)
data = data.drop('displayres', axis = 1)
data = data.drop('battery', axis = 1)
data = data[data.os != -1]
data = data[data.cpubrand != -1]
data = data[data.price != 0]
data = data[data.displaysize != 'Chính: 5.7, phụ: 2.05"']
data = data[data.displayWidth != -1]

print(data)
data.to_csv('processedsmartphoneinfo.csv', sep = ";")


     smartphonename   price  ram  rom displaysize  provider  os  cpubrand  \
0                13  15.990    4   64         5.8         0   0       0.0   
1                 0  34.790    3  256         5.8         0   1       1.0   
2                 0  34.790    3  256         5.8         0   1       1.0   
3                 0  29.990    3   64         5.8         0   1       1.0   
4                 0  29.990    3   64         5.8         0   1       1.0   
5                 0  28.790    3  256         5.5         0   1       1.0   
6                 0  25.790    2  256         4.7         0   1       1.0   
7                13  24.490    6  128         6.2         0   0       0.0   
8                13  24.490    6  128         6.2         0   0       0.0   
9                 0  23.990    3   64         5.5         0   1       1.0   
10                0  23.990    3   64         5.5         0   1       1.0   
11               13  23.490    6   64         6.2         0   0       0.0   